- 한국어 텍스트 카운트 벡터 생성은 기본적으로 영어와 동일하다.
- 차이가 있다면 형태소 분석기뿐이다.
- 한글에 대한 분석을 할 때는 CounterVectorizer가 제공하는 토크나이저를 쓸 수 없다.
- 따라서 KoNLPy의 형태소 분석기를 지정해서 CounterVectorizer 객체를 선언한다.


In [2]:
#load
import pandas as pd
df = pd.read_csv('./daum_movie_review.csv')
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [3]:
# 토크나이저를 사용해서 카운터 벡터 생성 
from sklearn.feature_extraction.text import CountVectorizer
daum_cv = CountVectorizer(max_features=1000)

daum_DTM = daum_cv.fit_transform(df.review)
print(daum_DTM[0])
print(daum_cv.get_feature_names_out()[:100])

  (0, 366)	1
  (0, 189)	1
['10점' '18' '1987' '1도' '1점' '1점도' '2시간' '2시간이' '2편' '5점' '6점' '7점' '8점'
 'cg' 'cg가' 'cg는' 'cg도' 'cg만' 'good' 'of' 'ㅋㅋ' 'ㅋㅋㅋ' 'ㅋㅋㅋㅋ' 'ㅎㅎ' 'ㅎㅎㅎ'
 'ㅜㅜ' 'ㅠㅠ' 'ㅠㅠㅠ' 'ㅡㅡ' '가는' '가는줄' '가면' '가서' '가슴' '가슴아픈' '가슴이' '가장' '가족'
 '가족과' '가족들과' '가족의' '가족이' '가지고' '간만에' '갈수록' '감독' '감독님' '감독은' '감독의' '감독이'
 '감동' '감동과' '감동도' '감동은' '감동을' '감동이' '감동입니다' '감동적' '감동적이고' '감동적인' '감사드립니다'
 '감사합니다' '감정이' '갑자기' '갔는데' '갔다가' '강철비' '강추' '강추합니다' '같고' '같네요' '같다' '같습니다'
 '같아' '같아요' '같은' '같은데' '같음' '같이' '개연성' '개연성이' '개인적으로' '거의' '겁나' '것도' '것은'
 '것을' '것이' '것이다' '겨울왕국' '결국' '결말' '결말이' '계속' '고맙습니다' '곤지암' '공포' '공포를'
 '공포영화' '관객']


In [4]:
# 위 결과는 '감동도', '감동과' 등 조사만 다르고 같은 의미의 단어가 많다 
#한글 형태소 분석기를 사용해 처리한다. 
from konlpy.tag import Komoran
Komoran = Komoran()

print('전체 형태소 결과:', Komoran.morphs(df.review[1]))
print('명사 추출 결과:', Komoran.nouns(df.review[1]))
print('품사 태그 결과:', Komoran.pos(df.review[1]))

전체 형태소 결과: ['몰입', '하', 'ㄹ', '수', '밖에', '없', '다', '.', '어렵', '게', '생각', '하', 'ㄹ', '필요', '없', '다', '.', '내', '가', '전투', '에', '참여', '하', 'ㄴ듯', '손', '에', '땀', '이남', '.']
명사 추출 결과: ['몰입', '수', '생각', '필요', '전투', '참여', '손', '땀', '이남']
품사 태그 결과: [('몰입', 'NNG'), ('하', 'XSV'), ('ㄹ', 'ETM'), ('수', 'NNB'), ('밖에', 'JX'), ('없', 'VA'), ('다', 'EF'), ('.', 'SF'), ('어렵', 'VA'), ('게', 'EC'), ('생각', 'NNG'), ('하', 'XSV'), ('ㄹ', 'ETM'), ('필요', 'NNG'), ('없', 'VA'), ('다', 'EF'), ('.', 'SF'), ('내', 'NP'), ('가', 'JKS'), ('전투', 'NNG'), ('에', 'JKB'), ('참여', 'NNG'), ('하', 'XSV'), ('ㄴ듯', 'EC'), ('손', 'NNG'), ('에', 'JKB'), ('땀', 'NNG'), ('이남', 'NNG'), ('.', 'SF')]


In [5]:
# 위 결과에서 필요없는 단어가 많다. 명사만 사용하면 너무 적다. 
# 조사나 특수문자 등만 제외하고 명사, 동사, 형용사만 선택한다. 

def my_tokenizer(doc):
    return[
        token
        for token, pos in Komoran.pos(doc)
        if pos in ['NNG', 'NNP', 'YY', 'VA']
    ]
print('토큰화 결과', my_tokenizer(df.review[1]))

토큰화 결과 ['몰입', '없', '어렵', '생각', '필요', '없', '전투', '참여', '손', '땀', '이남']


In [6]:
# 위 함수를 지정해서 객채 생성하고 fit_transform으로 카운트 벡터를 생성 
# CountVectorizer에서 토크나이저로 매번 my_tokenizer를 호출한다 
from sklearn.feature_extraction.text import CountVectorizer

daum_cv = CountVectorizer(max_features=1000, tokenizer=my_tokenizer)

# review 를 이용해 카운트 벡터를 학습하고 반환 
daum_DTM = daum_cv.fit_transform(df.review)

print(daum_cv.get_feature_names_out()[:100])

C:\Anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['!!' '!!!' '-1' '10분' '80년대' '가' '가볍' '가슴' '가족' '가치' '각본' '각색' '간만' '감'
 '감독' '감동' '감명' '감사' '감상' '감성' '감안' '감정' '감정이입' '감탄' '갑' '값' '강' '강림' '강요'
 '강철' '강하' '같' '개' '개그' '개봉' '개연' '개인' '객관' '거리' '거지' '거짓' '거짓말' '걱정' '건지'
 '겁' '게' '게임' '겨울' '겨울왕국' '결과' '결론' '결말' '결혼' '경찰' '경험' '계기' '고' '고맙'
 '고맙습니다' '고민' '고생' '고스' '고통' '곡' '곤지암' '곳' '공' '공감' '공포' '공포 영화' '공포감'
 '공포물' '과' '과거' '과장' '과정' '관객' '관계' '관람' '관심' '광고' '광주' '광주민주화운동' '광주사태'
 '괘' '괜찮' '교훈' '구성' '국가' '국내' '국민' '군' '군가' '군부' '군인' '군함도' '굿' '권선징악' '귀'
 '귀신']


In [8]:
# 희소도 확인 
print(repr(daum_DTM))
print(84188/(14725*1000))

<14725x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 84188 stored elements in Compressed Sparse Row format>
0.005717351443123939


In [9]:
# 단어 개수 확인 
for word, count in zip(daum_cv.get_feature_names_out(), daum_DTM[1].toarray()[0]):
    if count > 0:
        print(word, ":", count)

몰입 : 1
생각 : 1
손 : 1
어렵 : 1
없 : 2
전투 : 1
필요 : 1
